In [ ]:
library(tidyverse)
library(DBI)
library(RSQLite)

In [ ]:
soccer <- dbConnect(SQLite(), 
                 "../input/soccer/database.sqlite")

In [ ]:
##### John's code
library(reshape2)
library(data.table)
library(dplyr)
library(knitr)

In [ ]:
## list all tables
tables <- dbListTables(soccer)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]
lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- 
    dbGetQuery(conn=soccer, 
               statement=paste("SELECT * FROM '", 
                               tables[[i]], "'", sep=""))
}

# create dataframes
country <-  data.frame(lDataFrames[1])
league  <-  data.frame(lDataFrames[2])
match   <-  data.frame(lDataFrames[3])
player  <-  data.frame(lDataFrames[4])
team    <-  data.frame(lDataFrames[6])

# select relevant columns
country <- select(country, id, name) %>% 
  rename(country_id = id)  %>% 
  rename(country_name = name)   # use country_id as key for join
league  <- select(league, country_id, name) %>% 
  rename(league_name = name) # use country_id as key for join
match   <- select(match, id, country_id, league_id, season, 
                  stage, date, match_api_id, home_team_api_id, 
                  away_team_api_id, home_team_goal, away_team_goal, 
                  home_player_1, home_player_2, home_player_3, 
                  home_player_4, home_player_5, home_player_6, 
                  home_player_7, home_player_8, home_player_9, 
                  home_player_10, home_player_11, away_player_1, 
                  away_player_2, away_player_3, away_player_4, 
                  away_player_5, away_player_6, away_player_7, 
                  away_player_8, away_player_9, away_player_10, 
                  away_player_11, goal, shoton, shotoff, 
                  foulcommit, card, cross, corner, possession)
player  <- select(player,player_api_id, player_name) # use player_api_id as key for join
team    <- select(team, team_api_id, team_long_name, team_short_name) # use team_api_id as key for join

others = colnames(match)[1:11]
players = colnames(match)[12:33]

flatten <- melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id") %>% 
  mutate(team_api_id = 
           ifelse(grepl("home",variable),home_team_api_id, 
                  ifelse(grepl("away",variable),away_team_api_id,NA))) %>%  
  left_join(country, by = "country_id")    %>% 
  left_join(league,  by = "country_id")    %>%  
  left_join(team,    by = "team_api_id")   %>%
  left_join(player,  by = "player_api_id") %>% 
  separate(season, into = c("season_start","season_end"), sep = "/", convert = TRUE) 
head(flatten)

team_players <- select(flatten, season_start, season_end, 
                       country_name, league_name, player_api_id, 
                       team_api_id, team_long_name, team_short_name,
                       player_name)

team_players_2015 <- team_players %>% filter(season_start == 2015)
team_players_2014 <- team_players %>% filter(season_start == 2014)

In [ ]:
##### my code
player <- dbReadTable(soccer, 'Player')
player_attr <- dbReadTable(soccer, 'Player_Attributes')

player <- as.data.frame(player)
player_attr <- as.data.frame(player_attr)
player_attr$date <- substr(player_attr$date,1,10)

In [ ]:
# filter season date
player_attr_2014 <- subset(player_attr, 
                           date >= '2014-07-18' & date <= '2015-05-31')
player_attr_2015 <- subset(player_attr, 
                           date >= '2015-07-17' & date <= '2016-05-25')

In [ ]:
# remove negative values
player[player < 0] <- NA
player_attr_2014[player_attr_2014 < 0] <- NA
player_attr_2015[player_attr_2015 < 0] <- NA

In [ ]:
# merge values of one player to one
aggregate_process <- function(x) {
  if (is.numeric(x)) {mean(x)}
  else {x[1]}
}

player_attr_2014_aggregate <- aggregate(player_attr_2014, 
               by=list(player_attr_2014$player_api_id),
               aggregate_process)
player_attr_2014 <- 
  player_attr_2014_aggregate[2:ncol(player_attr_2014_aggregate)]

player_attr_2015_aggregate <- aggregate(player_attr_2015, 
                                        by=list(player_attr_2015$player_api_id),
                                        aggregate_process)
player_attr_2015 <- 
  player_attr_2015_aggregate[2:ncol(player_attr_2015_aggregate)]

In [ ]:
# remove outliers
# normalize
preprocess <- function(data) {
  for(i in 1:ncol(data)) {
    if (is.numeric(data[,i]) && !grepl("id",names(data)[i])) {
      mean <- mean(data[,i])
      sd <- sd(data[,i])
      data[,i] <- replace(data[,i],data[,i]>mean+2*sd,NA)
      data[,i] <- replace(data[,i],data[,i]<mean-2*sd,NA)
      data[,i] <- scale(data[,i])
    }
  }
  return(data)
}

player <- preprocess(player)
player_attr_2014 <- preprocess(player_attr_2014)
player_attr_2015 <- preprocess(player_attr_2015)

In [ ]:
# merge with players' basic info
player_2014 <- merge(player,player_attr_2014[,c(2:42)],
                     by=c('player_api_id','player_fifa_api_id'),
                     all.x = TRUE, all.y = TRUE)
player_2015 <- merge(player,player_attr_2015[,c(2:42)],
                     by=c('player_api_id','player_fifa_api_id'),
                     all.x = TRUE, all.y = TRUE)

In [ ]:
# filter only 3 leagues
player_name_2014 <- team_players_2014$player_name
player_2014 <- subset(player_2014,
                      player_name %in% player_name_2014)
player_name_2015 <- team_players_2015$player_name
player_2015 <- subset(player_2015,
                      player_name %in% player_name_2015)

In [ ]:
# change dummies
player_2014$left_foot <- ifelse(player_2014$preferred_foot == 'left', 1, 0)
player_2014$right_foot <- ifelse(player_2014$preferred_foot == 'right', 1, 0)
player_2015$left_foot <- ifelse(player_2015$preferred_foot == 'left', 1, 0)
player_2015$right_foot <- ifelse(player_2015$preferred_foot == 'right', 1, 0)

player_2014$attacking_work_rate <- replace(
  player_2014$attacking_work_rate,player_2014$attacking_work_rate=='None',NA)
player_2014$attack_high <- ifelse(player_2014$attacking_work_rate == 'high', 1, 0)
player_2014$attack_medium <- ifelse(player_2014$attacking_work_rate == 'medium', 1, 0)

player_2015$attacking_work_rate <- replace(
  player_2015$attacking_work_rate,player_2015$attacking_work_rate=='None',NA)
player_2015$attack_high <- ifelse(player_2015$attacking_work_rate == 'high', 1, 0)
player_2015$attack_medium <- ifelse(player_2015$attacking_work_rate == 'medium', 1, 0)

player_2014$defend_high <- ifelse(player_2014$defensive_work_rate == 'high', 1, 0)
player_2014$defend_medium <- ifelse(player_2014$defensive_work_rate == 'medium', 1, 0)
player_2015$defend_high <- ifelse(player_2015$defensive_work_rate == 'high', 1, 0)
player_2015$defend_medium <- ifelse(player_2015$defensive_work_rate == 'medium', 1, 0)

In [ ]:
full_league_stat_14 <- team_players_2014 %>% left_join(player_2014, by = 'player_api_id') %>%
    select(c('league_name', 'overall_rating')) %>%
    group_by_at(1) %>%
    summarise_each(funs(mean(., na.rm=T))) %>% 
    left_join(league, by = 'league_name') %>% 
    left_join(data.frame(lDataFrames[1]), by = c('country_id'='id'))
full_league_stat_14[2,4] = 'UK'
full_league_stat_14

In [ ]:
library(ggplot2)
require(maps)

# map of europe
some.eu.countries <- full_league_stat_14$name
eur <- map_data("world", region = some.eu.countries) %>% left_join(full_league_stat_14, by = c('region' = 'name'))
region.label <- eur %>% 
  group_by(region) %>%
  summarise_each(funs(mean), c(1:2))


ggplot(eur, aes(x = long, y = lat)) +
  geom_polygon(aes(group = group, fill = overall_rating))+
  geom_text(aes(label = region), data = region.label,  size = 3, hjust = 0.5)+
  coord_fixed(1.3) +
  theme(legend.position = "right") 